In [3]:
import tweepy 
import configparser
import pandas as pd
import os

In [4]:
#read configs
config = configparser.ConfigParser()
config.read('config.ini')

api_key         = config['twitter']['api_key']
api_key_secret  = config['twitter']['api_key_secret']

access_token        = config['twitter']['access_token']
access_token_secret = config['twitter']['access_token_secret']

bearer_token        = config['twitter']['bearer_token']

In [5]:
#Authenticate our account with the Twitter API

auth    = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api     = tweepy.API(auth)

# You can authenticate as your app with just your bearer token
client = tweepy.Client(bearer_token=bearer_token)

# If the authentication was successful, this should print the
# screen name / username of the account
print(api.verify_credentials().screen_name)

KLuthra_


In [40]:
words   = '#ad OR #sponsored OR #promoted -is:retweet from:verified lang:en'
tweets  = tweepy.Paginator(client.search_recent_tweets, query = words, 
                                tweet_fields = ['id', 'text', 'author_id', 'attachments', 'context_annotations',
                                                'created_at', 'entities', 'public_metrics'],
                                max_results  = 100).flatten(limit=100)



In [77]:
# function to display data of each tweet
def printtweetdata(n, ith_tweet):
        print()
        print(f"Tweet {n}")
        print(f"Tweet ID:  {ith_tweet[0]}")
        print(f"Tweet Text:{ith_tweet[1]}")
        print(f"Author ID: {ith_tweet[2]}")
        print(f"Created At:{ith_tweet[3]}")
        print(f"Domain ID: {ith_tweet[4]}")   
        print(f"Domain Name:{ith_tweet[5]}")   
        print(f"Domain Desc:{ith_tweet[6]}")   
        print(f"Entity ID:{ith_tweet[7]}")   
        print(f"Entity Name:{ith_tweet[8]}")   
        print(f"Public Metrics:{ith_tweet[9]}")
        print(f"Entities-Hashtags:{ith_tweet[10]}")    

In [93]:
#function to perform data extraction
def scrape(words, numtweet):

    # Creating Dataframe using pandas
    db = pd.DataFrame(columns= ['Tweet_ID',
                                'Tweet_Text',
                                'Author_ID',
                                'Created_At',
                                'DomainID', 
                                'Domain_Name',
                                'Domain_Desc', 
                                'Entity_ID', 
                                'Entity_Name', 
                                'Public_Metrics',
                                'Hashtags', 
                                ])
          
    # query   = '#ad OR #sponsored -is:retweet from:verified lang:en'
    tweets  = tweepy.Paginator(client.search_recent_tweets, query = words, 
                                tweet_fields = ['id', 'text', 'author_id', 'attachments', 'context_annotations',
                                                'created_at', 'entities', 'public_metrics'],
                                max_results  = 100).flatten(limit=numtweet)


    list_tweets = [tweet for tweet in tweets]

    i = 1

    for tweet in list_tweets:
                tweetid     = tweet.id
                text        = tweet.text
                authorid    = tweet.author_id
                createdTime = tweet.created_at

                domainID    = []
                domainNames = []
                domaindesc  = []
                entityID    = []
                entityNames = []
                for annotation in tweet.context_annotations:
                    domainID.append(annotation['domain']['id'])
                    domainNames.append(annotation['domain']['name'])
                    domaindesc.append(annotation['domain'].get('description', ''))
                    entityID.append(annotation['entity']['id'])
                    entityNames.append(annotation['entity']['name'])
                metrics     = tweet.public_metrics
                if 'hashtags' in tweet.entities:
                    hashtags= tweet["entities"]["hashtags"]
                else:
                    hashtags= []
              

    
    # Here we are appending all the
    # extracted information in the DataFrame
                ith_tweet       = [tweetid,text,authorid,createdTime,domainID,domainNames,domaindesc,
                entityID,entityNames,metrics,hashtags]
                db.loc[len(db)] = ith_tweet

    # Function call to print tweet data on screen
                printtweetdata(i, ith_tweet)
                i = i+1
    filename = 'scraped_tweets.csv'
    # we will save our database as a CSV file
    db.to_csv(filename)

In [94]:
 # number of tweets you want to extract in one run
numtweet = 100
scrape("#ad OR #sponsored OR #promoted from: verified", numtweet)
print('Scraping has completed!')


Tweet 1:
Tweet ID:  1609761436305346561
Tweet Text:RT @FrostNFT_: 🎁GIVEAWAY🎁

Welcome to Pudgy land!

🏆 1 Solhub Discord Invite
🏆 10 WL Spots

Entry:
1️⃣ Follow 
@PudgyFriends
 &amp;
@FrostNFT_…
Author ID: 1259737070957985797
Created At:2023-01-02 04:00:08+00:00
Domain ID: ['47', '131', '165']
Domain Name:['Brand', 'Unified Twitter Taxonomy', 'Technology']
Domain Desc:['Brands and Companies', 'A taxonomy of user interests. ', 'for individual and types of technology, e.g., food technology, 3D printing']
Entity ID:['1168499224297656321', '1168499224297656321', '1196446161223028736']
Entity Name:['Discord', 'Discord', 'Social media']
Public Metrics:{'retweet_count': 80, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}
Entities-Hashtags:[]

Tweet 2:
Tweet ID:  1609761432744374272
Tweet Text:RT @knithacker: From great yarns to crafty books to the latest gadgets &amp; accessories ... every day I update my Amazon storefront with the n…
Author ID: 712493579977887744
Created At:2023-01-02 